In [ ]:
# install google's VAD
!pip install webrtcvad

#install librosa and sox
# !apt-get install libsox-fmt-all
!apt-get install sox
!pip install sox
!pip install import_ipynb

In [ ]:
from google.colab import drive
import import_ipynb
import sys

drive.mount('/content/drive/',force_remount = False)
proj_dir_path = '/content/drive/MyDrive/Study_materials/Voice_disorder_detection_project/'
sys.path.append(proj_dir_path)
%cd $proj_dir_path

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/Othercomputers/My MacBook Pro/Study materials/Voice_disorder_detection_project


In [ ]:
## Imports

import numpy as np
import glob
import os
import torch
import torchaudio
import matplotlib.pyplot as plt
import soundfile as sf
from IPython.display import Audio
import pandas as pd
from scipy.io import wavfile
import re
from src.params.params import CommonParams as cfg
from src.params.params import webrtcvadParams as vad_params
from src.params.params import data_location
import shutil

import webrtcvad
import sox
import collections
import contextlib
import wave


print(torch.__version__)
print(torchaudio.__version__)

## Use cuda if available
torch.random.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

raw_voice_data = data_location['raw_data']
print(device)

VAD Wrapper funcs

In [1]:



def get_num_of_chunk_as_int(chunk):
  # receives a chunk's name in str and extracts the num as integer
  # expects the chunk name to be in the a specific format
  return (chunk.split(".")[0]).split("-")[1]

def get_wav_files(path='/content/'):
  return glob.glob(path + '/*.wav')

def read_wave(path):
    """Reads a .wav file.

    Takes the path, and returns (PCM audio data, sample rate).
    """
    with contextlib.closing(wave.open(path, 'rb')) as wf:
        num_channels = wf.getnchannels()
        assert num_channels == 1
        sample_width = wf.getsampwidth()
        assert sample_width == 2
        sample_rate = wf.getframerate()
        assert sample_rate in (8000, 16000, 32000, 48000)
        pcm_data = wf.readframes(wf.getnframes())
        return pcm_data, sample_rate


def write_wave(path, audio, sample_rate):
    """Writes a .wav file.

    Takes path, PCM audio data, and sample rate.
    """
    with contextlib.closing(wave.open(path, 'wb')) as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(sample_rate)
        wf.writeframes(audio)



class Frame(object):
    """Represents a "frame" of audio data."""
    def __init__(self, bytes, timestamp, duration):
        self.bytes = bytes
        self.timestamp = timestamp
        self.duration = duration


def frame_generator(frame_duration_ms, audio, sample_rate):
    """Generates audio frames from PCM audio data.

    Takes the desired frame duration in milliseconds, the PCM data, and
    the sample rate.

    Yields Frames of the requested duration.
    """
    n = int(sample_rate * (frame_duration_ms / 1000.0) * 2)
    offset = 0
    timestamp = 0.0
    duration = (float(n) / sample_rate) / 2.0
    while offset + n < len(audio):
        yield Frame(audio[offset:offset + n], timestamp, duration)
        timestamp += duration
        offset += n


def vad_collector(sample_rate, frame_duration_ms,
                  padding_duration_ms, vad, frames):
    """Filters out non-voiced audio frames.

    Given a webrtcvad.Vad and a source of audio frames, yields only
    the voiced audio.

    Uses a padded, sliding window algorithm over the audio frames.
    When more than 90% of the frames in the window are voiced (as
    reported by the VAD), the collector triggers and begins yielding
    audio frames. Then the collector waits until 90% of the frames in
    the window are unvoiced to detrigger.

    The window is padded at the front and back to provide a small
    amount of silence or the beginnings/endings of speech around the
    voiced frames.

    Arguments:

    sample_rate - The audio sample rate, in Hz.
    frame_duration_ms - The frame duration in milliseconds.
    padding_duration_ms - The amount to pad the window, in milliseconds.
    vad - An instance of webrtcvad.Vad.
    frames - a source of audio frames (sequence or generator).

    Returns: A generator that yields PCM audio data.
    """
    num_padding_frames = int(padding_duration_ms / frame_duration_ms)
    # We use a deque for our sliding window/ring buffer.
    ring_buffer = collections.deque(maxlen=num_padding_frames)
    # We have two states: TRIGGERED and NOTTRIGGERED. We start in the
    # NOTTRIGGERED state.
    triggered = False

    voiced_frames = []
    for frame in frames:
        is_speech = vad.is_speech(frame.bytes, sample_rate)

        sys.stdout.write('1' if is_speech else '0')
        if not triggered:
            ring_buffer.append((frame, is_speech))
            num_voiced = len([f for f, speech in ring_buffer if speech])
            # If we're NOTTRIGGERED and more than 90% of the frames in
            # the ring buffer are voiced frames, then enter the
            # TRIGGERED state.
            if num_voiced > 0.9 * ring_buffer.maxlen:
                triggered = True
                sys.stdout.write('+(%s)' % (ring_buffer[0][0].timestamp,))
                # We want to yield all the audio we see from now until
                # we are NOTTRIGGERED, but we have to start with the
                # audio that's already in the ring buffer.
                for f, s in ring_buffer:
                    voiced_frames.append(f)
                ring_buffer.clear()
        else:
            # We're in the TRIGGERED state, so collect the audio data
            # and add it to the ring buffer.
            voiced_frames.append(frame)
            ring_buffer.append((frame, is_speech))
            num_unvoiced = len([f for f, speech in ring_buffer if not speech])
            # If more than 90% of the frames in the ring buffer are
            # unvoiced, then enter NOTTRIGGERED and yield whatever
            # audio we've collected.
            if num_unvoiced > 0.9 * ring_buffer.maxlen:
                sys.stdout.write('-(%s)' % (frame.timestamp + frame.duration))
                triggered = False
                yield b''.join([f.bytes for f in voiced_frames])
                ring_buffer.clear()
                voiced_frames = []
    if triggered:
        sys.stdout.write('-(%s)' % (frame.timestamp + frame.duration))
    sys.stdout.write('\n')
    # If we have any leftover voiced audio when we run out of input,
    # yield it.
    if voiced_frames:
        yield b''.join([f.bytes for f in voiced_frames])

In [ ]:
def webrtcvad_usage(aggressive=0, audio_path=None,frame_duration=30,padding_duration=300,min_chunk_length = 0.5):
    vad = webrtcvad.Vad(aggressive)
    audio, sample_rate  = read_wave(audio_path)
    assert sample_rate == cfg.TARGET_SAMPLE_RATE, f"Sample rate is not 16k hz. {sample_rate}"
    frames = frame_generator(frame_duration, audio, sample_rate)
    frames = list(frames)
    segments = vad_collector(sample_rate, frame_duration, padding_duration, vad, frames)



  #  sample_rate - The audio sample rate, in Hz.
  #   frame_duration_ms - The frame duration in milliseconds.
  #   padding_duration_ms - The amount to pad the window, in milliseconds.
  #   vad - An instance of webrtcvad.Vad.
  #   frames - a source of audio frames (sequence or generator).
    i = 0
    for j,segment in enumerate(segments):
      path = '/content/chunk-%002d.wav' % (i,)
      # print(' Writing %s' % (path,))
      len_sec = len(segment)/(sample_rate*2)
      # if( i < 10):
      if((len_sec >= vad_params.MIN_CHUNK_LENGTH) ):
        write_wave(path, segment, sample_rate)
        # print(f'\n phone Chunk {i} length is {len_sec} seconds')
        i = i+1
      # else:
        # print(f'\n too short! Chunk {i} length is {len_sec} seconds')
    return segments

In [ ]:
def audio_resample_mono(pat_path=""):
  ## Given an audio path, resamples, mono-channels and produces a .wav file
  pat_num_str = pat_path.split(' ')[1].split('.')[0]
  pat_num = int(pat_num_str)
  if(not pat_path or pat_num == -1):
    print("Invalid patient given")
    return
          ## Sox usage - resample and monochannel
  # file name can start with capital P or lowercase p, we take the monochannel
  file_spaces = file.replace(" ", "\\ ")
  !sox {file_spaces} -c 1 /content/pat_{pat_num}_mono.wav
  # resample to 16k
  !sox /content/pat_{pat_num}_mono.wav -r 16000 /content/pat_{pat_num_str}_mono_16k.wav
  # delete older file
  !rm /content/pat_{pat_num}_mono.wav
  mono_16k_wav_path = f'/content/pat_{pat_num_str}_mono_16k.wav'
  return mono_16k_wav_path , pat_num_str

def audio_split_into_chunks(mono_16k_path = "",pat_num_str = "-1",aggressive=3,frame_duration=10,padding_duration=150):

  # google VAD usage
  assert (mono_16k_path), f'Sox failed to produce .wav file'
  webrtcvad_usage(aggressive=aggressive, audio_path=mono_16k_path,
                  frame_duration=frame_duration,
                  padding_duration=padding_duration,
                  min_chunk_length = vad_params.MIN_CHUNK_LENGTH)
  # Delete the raw audio file
  !rm {mono_16k_path}
  chunk_list = [None] * 200 # Assuming no audio is being split into more than 200 chunks
  cbn=sox.Combiner()
  amount_of_chunks = len(get_wav_files())
  if(amount_of_chunks > 0 ):
    for chunk in glob.glob('/content/chunk-??.wav'):
      chunk_num = int(get_num_of_chunk_as_int(chunk))
      if(chunk_num < 200):
        chunk_list[chunk_num] = chunk
      else:
        print("Number of audio chunks overflowed")
  else:
    print(f"Recording {pat_num_str} is too noisy to be split")
    return [None]
  return chunk_list

def combine_chunks(pat_num_str = "-1", chunk_list=[],save_to_drive = False):
  assert isinstance(chunk_list,list),f'no chunk list was given'
  assert len(chunk_list) > 0 ,f'chunk list is empty'
  # # Combine all chunks into a single file
  cbn=sox.Combiner()
  chunk_list = [x for x in chunk_list if x is not None] # Filter out all the None's
  print(chunk_list)
  cbn.build(chunk_list,'/content/chunks_concatenated.wav','concatenate')
  !rm /content/chunk-**.wav
  !rm /content/chunk-***.wav

  try:
    if(save_to_drive):
      source_path = '/content/chunks_concatenated.wav'
      name = "patient_" + pat_num_str + ".wav"
      destination_path = os.path.join(data_location['preprocessed_data'], name)
      # Move and rename the file
      print("\n Saving cleaned audio to ",destination_path)
      shutil.move(source_path, destination_path)
  except Exception as e:
    print("Something happened:", e)

In [ ]:
def clean_audio(pat_path = "",save_to_drive = False):
  assert (pat_path) , f'Invalid patient path given'
  pat_num_str = re.findall(r'patient\\ (\d+)', pat_path.lower())[0]
  aggressive = vad_params.AGGRESSIVE
  frame_duration= vad_params.FRAME_DURATION
  padding_duration= vad_params.PADDING_DURATION

  mono_16k_wav_path , pat_num_str = audio_resample_mono(pat_path=pat_path)
  chunk_list = audio_split_into_chunks(mono_16k_path = mono_16k_wav_path,
                                       pat_num_str = pat_num_str,
                                       aggressive=aggressive,
                                       frame_duration=frame_duration,
                                      padding_duration=padding_duration)
  combine_chunks(pat_num_str = pat_num_str, chunk_list=chunk_list,save_to_drive = save_to_drive)

In [ ]:
class clean_all_audio():
  def __init__(self, save_to_drive= False):
    for file in glob.glob(data_location['raw_data'] + "/*.wav"):
      file_spaces = file.replace(" ", "\\ ")
      clean_audio(pat_path=file_spaces,save_to_drive=save_to_drive)